# 🤖 AI-Powered HR Assistant for Nestlé
This project builds a chatbot that can answer questions about Nestlé's HR policy using OpenAI's GPT-3.5 Turbo, LangChain, ChromaDB, and Gradio.

In [5]:
# ✅ Step 1: Install Required Libraries
!pip install openai langchain chromadb gradio pypdf tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import openai
print(openai.__version__)

0.28.1


In [6]:
import os
key = print(os.environ.get("OPENAI_API_KEY"))

voc-6977049751122724181614685d092355e3c9.03283979


In [9]:
import langchain
print("LangChain version:", langchain.__version__)

LangChain version: 0.3.26


In [ ]:
pip install -U langchain-community

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import openai

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]
# Test call
print(get_completion("Your query"))

Hello! How can I assist you today?


In [5]:
print(get_completion("Hello, what is AI?"))

AI stands for artificial intelligence, which refers to the simulation of human intelligence processes by machines, especially computer systems. AI technologies enable machines to learn from experience, adapt to new inputs, and perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI is a rapidly evolving field with applications in various industries, including healthcare, finance, transportation, and entertainment.


In [1]:
# ✅ Step 3: Load and Split PDF Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
import re

# Load and clean text
loader = PyPDFLoader("HR_Policy_Nestle.pdf")
docs = loader.load()
raw_text = "\n".join(doc.page_content for doc in docs)

# Fix line breaks and spacing
clean_text = re.sub(r"(?<![A-Z])\n(?![A-Z])", " ", raw_text)
clean_text = re.sub(r"\s{2,}", " ", clean_text).strip()

# Updated section titles
section_titles = [
    "INTRODUCTION",
    "A SHARED RESPONSIBILITY",
    "JOINING NESTLÉ",
    "EMPLOYMENT AND WORKING CONDITIONS",
    "TOTAL REWARDS",
    "TRAINING AND LEARNING",
    "TALENT, DEVELOPMENT AND PERFORMANCE MANAGEMENT",
    "EMPLOYEE RELATIONS",
    "A FLEXIBLE AND DYNAMIC ORGANISATION"
]

# Split when one of the section titles appears (even inline)
split_pattern = r"(?=" + "|".join(re.escape(t) for t in section_titles) + r")"
parts = re.split(split_pattern, clean_text)

# Group into Document chunks (skip empty)
final_chunks = []
for i, part in enumerate(parts):
    part = part.strip()
    if any(part.startswith(title) for title in section_titles):
        title = next(t for t in section_titles if part.startswith(t))
        final_chunks.append(Document(page_content=part, metadata={"section_title": title}))

# Output
print(f"✅ Created {len(final_chunks)} properly split chunks.")
for i, chunk in enumerate(final_chunks[:9]):
    print(f"\n🧩 Chunk {i+1}: {chunk.metadata['section_title']}\n{'-'*40}")
    print(chunk.page_content[:3000])


✅ Created 9 properly split chunks.

🧩 Chunk 1: INTRODUCTION
----------------------------------------
INTRODUCTION At Nestlé, we recognize that our employees are the key to our success and nothing can be achieved without their engagement. This document encompasses the guidelines which constitute a solid basis for effective Human Resources Management throughout the Nestlé Group around the world. It explains to all Nestlé employees the vision and mission of the Human Resources function and illustrates every aspect of the Nestlé employee lifecycle. The Nestlé Management and Leadership Principles inspire all the Nestlé employees in their actions and in their dealings with others. The Corporate Business Principles refer to all the basic principles which Nestlé endorses and subscribes to on a worldwide basis. Both these documents are the pillars on which the present policy has been built. The implementation of this policy will be inspired by sound judgement, compliance with local market laws 

In [12]:
!pip install pysqlite3-binary

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
import pysqlite3
sys.modules["sqlite3"] = pysqlite3
from langchain_community.vectorstores import Chroma

In [14]:
pip install chromadb==0.4.22

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -U langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ✅ Step 4: Create Vector Store
from langchain_openai import OpenAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# ✅ Step 1: Create embedding model
embedding_model = OpenAIEmbeddings()

# ✅ Step 2: Create Chroma vector store (in memory)
vectordb = Chroma.from_documents(
    documents=final_chunks,     # the cleaned and split chunks
    embedding=embedding_model,
    collection_name="nestle_hr_policy",
    persist_directory=None      # or use "./chroma_store" if you want to persist
)

# ✅ Step 3: Test retrieval
retriever = vectordb.as_retriever()
docs = retriever.invoke("What is Nestlé's policy on employee benefits and rewards?")

# ✅ Step 4: Preview retrieved answer
print("🔍 Retrieved document:\n", docs[0].page_content[:3000])


🔍 Retrieved document:
 TOTAL REWARDS Attracting new hires and keeping current employees engaged is not only about remuneration and benefits based on solid performance. It is also about the hard earned value and trust that our name brings to those who work with us; the relationships with our line managers and fellow workers; recognition and experiences enjoyed while working for a diverse global company; and possibilities to learn and grow. These are as a whole, the Total Rewards we receive. Nestlé, therefore, focuses on Fixed Pay, Variable Pay, Benefits, Personal Growth and Development and Work Life Environment as the key elements that define Total Rewards. In the spirit of developing a high performance culture, those elements need to correspond to what is valued by employees in each and every market, and which demonstrate how Nestlé is committed to giving each employee the opportunity to grow, evolve and contribute. Nestlé Total Rewards programmes must be established within the social 

In [5]:
# ✅ Step 5: Create Retrieval QA System
# ✅ Imports
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# ✅ Step 1: Define the refined prompt template
refined_general_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a professional and helpful HR Assistant for Nestlé. You answer employee questions based only on the official HR policy context provided.

🎯 Instructions:
- Be accurate, respectful, and clear.
- Do not guess or generate content beyond the provided context.
- If the answer is not clearly present in the context, respond without false and fake information
---
📚 HR Policy Context:
{context}

❓ User Question:
{question}

💬 Answer:"""
)

# ✅ Step 2: Set up the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ✅ Step 3: Create Retrieval QA chain with custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": refined_general_prompt}
)

# ✅ Step 4: Test it out
query = "What types of rewards and benefits does Nestlé offer?"
result = qa_chain.invoke(query)

# ✅ Step 5: Display the result
print("💬 Answer:\n", result['result'])
print("\n📚 Source Document:\n", result["source_documents"][0].page_content[:3000])


💬 Answer:
 Nestlé offers Total Rewards which include Fixed Pay, Variable Pay, Benefits, Personal Growth and Development, and Work Life Environment as key elements that define the rewards and benefits provided to employees. These programs are established within the social and legal framework of each country and with respect to applicable collective agreements.

📚 Source Document:
 TOTAL REWARDS Attracting new hires and keeping current employees engaged is not only about remuneration and benefits based on solid performance. It is also about the hard earned value and trust that our name brings to those who work with us; the relationships with our line managers and fellow workers; recognition and experiences enjoyed while working for a diverse global company; and possibilities to learn and grow. These are as a whole, the Total Rewards we receive. Nestlé, therefore, focuses on Fixed Pay, Variable Pay, Benefits, Personal Growth and Development and Work Life Environment as the key elements th

In [32]:
# ✅ Step 6.1: Test 1 with Sample Query
query = "What is the leave policy at Nestlé?"
response = qa_chain.invoke(query)
# Step 4: Output the result
print("💬 Answer:\n", response['result'])

# Optional: Show source content
print("\n📚 Source Document:\n", response["source_documents"][0].page_content[:3000])

💬 Answer:
 The HR policy context provided does not specifically mention the leave policy at Nestlé. For detailed information on the leave policy, I recommend reaching out to your HR department or referring to the official company handbook or policies.

📚 Source Document:
 EMPLOYMENT AND WORKING CONDITIONS We are committed to providing our employees all over the world with good working conditions, a safe and healthy work environment, and flexible employment possibilities that support a better balance of private and professional life consistent with our ambition as a leading Nutrition, Health and Wellness Company. As such, we provide flexible working conditions whenever possible and encourage our employees to have outside interests especially community involvement. Those with line management responsibilities are required to take personal ownership of safety and health within their area of responsibility and are encouraged to develop their capability in this area. Nestlé’s commitment howe

In [11]:
# ✅ Step 6.2: Test 2 with Sample Query
query = "What is the recruitment policy at Nestlé"
response = qa_chain.invoke(query)
# Step 4: Output the result
print("💬 Answer:\n", response['result'])

# Optional: Show source content
print("\n📚 Source Document:\n", response["source_documents"][0].page_content[:3000])

💬 Answer:
 The recruitment policy at Nestlé focuses on hiring employees with personal attitudes and professional skills that align with the company's values and culture. Only relevant skills, experience, and adherence to Nestlé principles are considered in the hiring process. No consideration is given to a candidate's origin, nationality, religion, race, gender, disability, sexual orientation, or age. The decision to hire a candidate is ultimately made by the responsible manager, supported by the HR team.

📚 Source Document:
 JOINING NESTLÉ The long-term success of the Company depends on its capacity to attract, retain and develop employees able to ensure ongoing and sustainable growth. This is a primary responsibility of all managers. The Nestlé policy is to hire employees with personal attitudes and professional skills enabling them to develop a long-term relationship with the Company. Therefore, special attention will be paid to ensure there is a strong alignment between a candidate

In [33]:
# ✅ Step 6.3: Test 3 with Sample Query
query = "Does Nestlé use recruitment tools?"
response = qa_chain.invoke(query)
# Step 4: Output the result
print("💬 Answer:\n", response['result'])

# Optional: Show source content
print("\n📚 Source Document:\n", response["source_documents"][0].page_content[:3000])

💬 Answer:
 Yes, Nestlé may use adequate recruitment tools to improve the hiring process. However, the decision to hire a candidate ultimately remains in the hands of the responsible manager, supported by the HR team.

📚 Source Document:
 JOINING NESTLÉ The long-term success of the Company depends on its capacity to attract, retain and develop employees able to ensure ongoing and sustainable growth. This is a primary responsibility of all managers. The Nestlé policy is to hire employees with personal attitudes and professional skills enabling them to develop a long-term relationship with the Company. Therefore, special attention will be paid to ensure there is a strong alignment between a candidate’s values and the Nestlé culture. Only relevant skills and experience and adherence to the Nestlé principles will be considered in employing a person. No consideration will be given to a candidate’s origin, nationality, religion, race, gender, disability, sexual orientation or age. Whilst adeq

In [37]:
pip install --upgrade gradio

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 33.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 5.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 1.7 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 8.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 50.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.2 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 932.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.3 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-lo

In [38]:
import gradio as gr
print("Gradio version:", gr.__version__)

Gradio version: 4.7.1


In [ ]:
import gradio as gr

# ✅ QA Bot function for Gradio ChatInterface
def qa_bot(messages, history):
    try:
        # Handle messages as list of dicts or string fallback
        if isinstance(messages, str):
            query = messages
        elif isinstance(messages, list) and isinstance(messages[-1], dict):
            query = messages[-1].get("content", "")
        else:
            return "❌ Sorry, I couldn't understand your question format."

        if not query.strip():
            return "❗ Please enter a valid question."

        # Get response from LangChain QA system
        result = qa_chain.invoke(query)
        answer = result.get("result", "").strip()

        # Gracefully fallback if nothing found
        if not answer:
            answer = "🤖 Sorry, I couldn't find a specific answer based on the provided documents."

        # Show only the most relevant source
        source_chunks = result.get("source_documents", [])
        if source_chunks:
            sources = f"""
<details>
  <summary>📚 Click to view source</summary>
  <pre>{source_chunks[0].page_content[:300].strip()}...</pre>
</details>
"""
        else:
            sources = "\n\n📚 *No document source available.*"

        return f"{answer}\n\n{sources}"

    
    except Exception as e:
        return f"❌ An error occurred while processing your request:\n`{str(e)}`"

# ✅ Launch with Gradio ChatInterface
chat_ui = gr.ChatInterface(
    fn=qa_bot,
    title="📘 Nestlé HR Assistant",
    description="Ask any question based on Nestlé’s HR policy documents. Powered by AI.",
    theme="soft",
     type="messages",
    chatbot=gr.Chatbot(
        type="messages",
        label="Chatbot",
        show_copy_button=True
    ),
    examples=[
        "What are the employee benefits at Nestlé?",
        "Tell me about Nestlé’s training and development programs.",
        "How does Nestlé support flexible working?",
        "What is Nestlé's policy on performance evaluation?",
        "Does Nestlé offer remote work options?"
    ]
)

chat_ui.launch(share=True)
